In [21]:
import os
import xml.etree.ElementTree as ET
from transformers import GPT2Tokenizer
import regex
import pickle

In [6]:
def load_all_files(directory):
    all_texts = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".xml"):
            file_path = os.path.join(directory, file_name)
            all_texts.extend(load_single_file(file_path))
    return all_texts

In [5]:
def load_single_file(file_path):
    texts = []
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Navigate to the <body> and <p> tags
        body = root.find(".//body")
        if body:
            for paragraph in body.findall(".//p"):
                if paragraph.text:
                    texts.append(paragraph.text.strip())
    except ET.ParseError as e:
        print(f"Error parsing {file_path}: {e}")
    return texts

In [ ]:
def clean_text(text):
    # Replacing multiple spaces with one
    text = regex.sub(r"\s+", " ", text)
    # Keep Devanagari and spaces
    text = regex.sub(r"[^\p{Devanagari}\s]", "", text)
    # Convert to lowercase
    text = text.lower().strip()
    return text

def clean_dataset(dataset):
    return [clean_text(text) for text in dataset]

In [ ]:
def tokenize_dataset(cleaned_dataset):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    
    # Set the padding token to be the same as eos_token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Tokenize the cleaned dataset
    tokenized_data = [tokenizer.encode(text, truncation=True, padding="max_length", max_length=512) for text in cleaned_dataset]
    
    return tokenized_data

In [ ]:
if __name__ == "__main__":
    directory = "Maithili"
    
    raw_texts = load_all_files(directory)

    cleaned_texts = clean_dataset(raw_texts)

    print("First cleaned sample:", cleaned_texts[0])

First cleaned sample: हुनक बेसी भाग महत्त्वपूर्ण कथा सब माटिसँ एहि तरहेँ जुड़ल अछि जे ओहि कथाक पात्र सब क्षेत्रीय नहि रहि जाइछ अवं पाठक अनुभव करैछ जे क्षेत्र और वातावर्णक किंचित परिवर्तनक पश्चात् ओ ग्रामीण भारतक कोनो भागक कथा भ सकैछ एक साधारण मनुष्य मृत्यु सँ संघर्ष करैत और अपन वर्तमान समयकेँ सार्थक करबाक प्रयासमे आदर्शरूप बनि जाइछ पौषी लक्ष्मी देवी लक्ष्मी जिनक आराधना पौष मासमे कैल जाइछक मुकुन्द पाल वैह आदिरूपक महत्ताकेँ प्राप्त करैछ ओ एक वृद्ध कृषक अछि आ अपन क्षेत्रमे सभसँ नीक अछि ई सन् १९४४ ई थिक बंगालक भयंकर चक्रवात और अकालक बादक साल एहि साल खेत सभ धानसँ भरल अछि जकरा शीघ्रहि काटब आवश्यक अछि दोसर विश्वयुद्ध चलि रहल अछि तैं खेतिहर मजदूर मुश्किलसँ भेटैछ और ओहि क्षेत्रक खेतिहर मजदूर सब विभिन्न युद्धनिर्माण कार्यमे नियुक्त भ गेल अछि पाकल और स्वर्णिम धान मुकुन्दक लेल लक्ष्मी अछि सम्पत्तिक प्रतीक मात्र पराक्रमी लक्ष्मीकेँ प्राप्त क सकैछ मुकुन्दकेँ अपन शक्ति क्षीणताक अनुभव होइछ अपन समयक प्रसिद्ध फसल कटनिहार एहि यथार्थकेँ स्वीकार नहि करैछ जे ओ वृद्धक संगहि अशक्त सेहो भ गेल अछि ओ ताकत जुटैबा

In [ ]:
tokenized_texts = tokenize_dataset(cleaned_texts)

print("First tokenized sample:", tokenized_texts[0])

First tokenized sample: [11976, 117, 24231, 223, 11976, 101, 11976, 243, 28225, 105, 24231, 229, 11976, 116, 24231, 222, 28225, 255, 48077, 11976, 245, 28225, 106, 11976, 117, 11976, 97, 24231, 235, 11976, 97, 24231, 235, 11976, 113, 11976, 103, 24231, 224, 11976, 108, 24231, 235, 11976, 96, 28225, 243, 11976, 98, 48077, 28225, 116, 11976, 105, 28225, 106, 48077, 11976, 253, 11976, 123, 11976, 116, 11976, 223, 28225, 237, 11976, 117, 11976, 123, 28225, 97, 11976, 108, 11976, 117, 24231, 229, 11976, 223, 28225, 250, 24231, 223, 11976, 94, 11976, 120, 11976, 110, 28225, 227, 11976, 249, 11976, 123, 28225, 250, 24231, 229, 28225, 241, 11976, 117, 11976, 123, 28225, 243, 11976, 98, 48077, 11976, 243, 28225, 103, 48077, 11976, 97, 24231, 235, 11976, 108, 28225, 116, 11976, 105, 28225, 243, 24231, 235, 11976, 115, 24231, 229, 11976, 97, 24231, 235, 11976, 108, 24231, 222, 11976, 107, 28225, 101, 11976, 117, 11976, 123, 28225, 108, 11976, 117, 11976, 123, 28225, 250, 48077, 11976, 229, 11976,

In [ ]:
with open("tokenized_data.pkl", "wb") as f:
    pickle.dump(tokenized_texts, f)

with open("tokenized_data.pkl", "rb") as f:
    loaded_tokenized_data = pickle.load(f)
    print("Loaded tokenized sample:", loaded_tokenized_data[0])

Loaded tokenized sample: [11976, 117, 24231, 223, 11976, 101, 11976, 243, 28225, 105, 24231, 229, 11976, 116, 24231, 222, 28225, 255, 48077, 11976, 245, 28225, 106, 11976, 117, 11976, 97, 24231, 235, 11976, 97, 24231, 235, 11976, 113, 11976, 103, 24231, 224, 11976, 108, 24231, 235, 11976, 96, 28225, 243, 11976, 98, 48077, 28225, 116, 11976, 105, 28225, 106, 48077, 11976, 253, 11976, 123, 11976, 116, 11976, 223, 28225, 237, 11976, 117, 11976, 123, 28225, 97, 11976, 108, 11976, 117, 24231, 229, 11976, 223, 28225, 250, 24231, 223, 11976, 94, 11976, 120, 11976, 110, 28225, 227, 11976, 249, 11976, 123, 28225, 250, 24231, 229, 28225, 241, 11976, 117, 11976, 123, 28225, 243, 11976, 98, 48077, 11976, 243, 28225, 103, 48077, 11976, 97, 24231, 235, 11976, 108, 28225, 116, 11976, 105, 28225, 243, 24231, 235, 11976, 115, 24231, 229, 11976, 97, 24231, 235, 11976, 108, 24231, 222, 11976, 107, 28225, 101, 11976, 117, 11976, 123, 28225, 108, 11976, 117, 11976, 123, 28225, 250, 48077, 11976, 229, 11976